In [5]:
import numpy as np

# Constantes físicas básicas
c_na = 6.02214179e23  # Constante de Avogadro
c_mu = 1.660538921e-27  # Massa atômica unitária
c_kb = 1.3806488e-23  # Constante de Boltzmann
c_pi = 3.14159265359  # Pi

# Modelo do argônio
sigma = 3.41e-10  # Sigma
epsilon = 119.8 * c_kb  # Epsilon
rc = 3.2 * sigma  # Distância de corte do potencial Lennard-Jones
m = 39.944 * c_mu  # Massa do átomo de argônio

# Definição do sistema estudado
n = 500  # Número de partículas na região de simulação
p = np.zeros((3, n))  # Posições das partículas
teplota = 100.0  # Temperatura
beta = 1.0 / (teplota * c_kb)  # Beta
h = np.zeros(3)  # Comprimentos das arestas da célula cúbica de simulação

# Parâmetros da mudança de Monte Carlo
max_posun = 0.2 * sigma  # Máximo de deslocamento

# Frações de passos aceitos em Monte Carlo
zp_pokusu_p = 0.0
zp_prijato_p = 0.0

# Variáveis para o cálculo do perfil de densidade
hp_n = 100
hp_sum = np.zeros(2 * hp_n + 1)
hp_num = 0



def posun_nahodna_particula():
    """Passo de Monte Carlo para mover uma partícula aleatória"""
    global zp_pokusu_p, zp_prijato_p
    zp_pokusu_p += 1.0

    # Escolher uma partícula aleatória
    c = np.random.randint(0, n)

    # Gerar um deslocamento aleatório para a partícula
    pz = p[:, c] + (np.random.rand(3) - 0.5) * max_posun

    # Aplicar as condições de contorno periódicas
    for i in range(3):
        if pz[i] > 0.5 * h[i]:
            pz[i] -= h[i]
        if pz[i] < -0.5 * h[i]:
            pz[i] += h[i]

    # Calcular a mudança na energia
    e = 0
    ez = 0
    for i in range(n):
        if i == c:
            continue
        e += e_ij(p[:, c], p[:, i], h)
        ez += e_ij(pz, p[:, i], h)

    # Aplicar o critério de aceitação do passo de Monte Carlo
    if np.random.rand() < np.exp(-beta * (ez - e)):
        p[:, c] = pz
        zp_prijato_p += 1.0

def e_pot(p, h):
    """Cálculo da energia potencial total"""
    e_pot = 0
    for i in range(n - 1):
        for j in range(i + 1, n):
            e_pot += e_ij(p[:, i], p[:, j], h)
    return e_pot

def e_ij(p1, p2, h):
    """Cálculo da energia potencial entre dois pares de partículas"""
    r = p2 - p1
    d2 = np.sum((0.5 * h - np.abs(0.5 * h - np.abs(r))) ** 2)

    if d2 < rc**2:
        e_ij = 4 * epsilon * ((sigma**2 / d2) ** 6 - (sigma**2 / d2) ** 3)
    else:
        e_ij = 0

    return e_ij

# Inicialização do gerador de números aleatórios
np.random.seed()

print('Iniciando a simulação do líquido na região cúbica')

# Comprimento das arestas da região cúbica, correspondente à densidade da fase líquida
h = [(n * m / 1400) ** (1.0 / 3)] * 3

# Teste de validade do comprimento do corte
if rc > 0.5 * h[0]:
    print('A aresta da região de simulação é muito curta ou rc é muito grande')
    exit()

# Geração da configuração inicial
# Posições aleatórias das partículas
p = np.random.rand(3, n)
p[0, :] = p[0, :] * h[0]
p[1, :] = p[1, :] * h[1]
p[2, :] = p[2, :] * h[2]

# Zerar os acumuladores para o perfil de densidade
hp_sum.fill(0)
hp_num = 0

# Simulação de Monte Carlo
for i in range(1, 10000001):
    zp_pokusu_p = 0
    zp_prijato_p = 0

    for j in range(1, 10001):
        posun_nahodna_particula()

    print(f"i={i}, e_pot={e_pot(p, h) / n * c_na:.3f}, zp_p={zp_prijato_p / zp_pokusu_p:.3f}")

    # Expansão da região de simulação após um equilíbrio razoável do líquido
    if i == 100:
        h[0] = 3 * h[0]
        print('A região de simulação foi expandida, começando a simulação da interface de fase')

    # Cálculo do perfil de densidade
    if i > 100:
        calc_hustotni_profil()

    # Armazenamento ocasional dos resultados do perfil de densidade e configuração para visualização
    if i % 100 == 0:
        np.savetxt('Simulacao6_hp.txt', [(j * h[0] / (2 * hp_n), m * hp_sum[j] / hp_num / (h[0] / (2 * hp_n) * h[1] * h[2])) for j in range(-hp_n, hp_n + 1)])
        
        with open('Simulacao6_konfiguracao.xyz', 'a') as f:
            f.write(f"{n}\n")
            f.write('Simulação da interface de fase do argônio\n')
            for j in range(n):
                f.write(f"Ar {p[0, j] * 1e10} {p[1, j] * 1e10} {p[2, j] * 1e10}\n")

    # Reinicialização do cálculo do perfil de densidade após atingir o equilíbrio
    if i == 1000:
        hp_sum.fill(0)
        hp_num = 0

# Funções auxiliares

def calc_hustotni_profil():
    """Cálculo do perfil de densidade"""
    x_cm = np.sum(p[0, :]) / n  # Coordenada x do centro de massa

    # Atualização do acumulador de histogramas
    for c in range(n):
        dx = p[0, c] - x_cm
        if dx > 0.5 * h[0]:
            dx = dx - h[0]
        if dx < -0.5 * h[0]:
            dx = dx + h[0]

        s = round(dx * hp_n / (0.5 * h[0]))
        if -hp_n <= s <= hp_n:
            hp_sum[s] += 1

    global hp_num
    hp_num += 1



Iniciando a simulação do líquido na região cúbica


TypeError: can't multiply sequence by non-int of type 'float'